# Detailed Analysis (Bid by Bid, Card by Card)

This is a tutorial of how to do a detailed analysis of a played board.

The engine looks at the bidding and play as it originally happened, and does an analysis for every bid and every card played.

The analysis is not just a double-dummy analysis for the exact current layout (like if you would press the "GIB" button on BBO). Instead, it's an analysis over many different possible layouts (samples).

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from analysis import CardByCard
from util import parse_lin, display_lin
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies



Instructions for updating:
non-resource variables are not supported in the long term


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/contract-193200
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_V2-3125000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_

In [3]:
# we specify all the information about a board
# (it's quite tedious to enter every single thing by hand here,
# later we'll have an example of how you can give it a board played on BBO)

dealer = 'N'
vuln = [False, False]  # fist element is NS, second element is EW

hands = ['87.84.T7.AKJ9764','JT53.QT.AJ6542.8','KQ96.K9.Q83.QT32','A42.AJ76532.K9.5']

auction = ['3C', 'PASS', '4C', '4H', 'PASS', 'PASS', 'PASS']

play = ['CA', 'C8', 'C3', 'C5',
    'S8', 'ST', 'SQ', 'SA',
    'DK', 'DT', 'D6', 'D3',
    'D9', 'D7', 'DA', 'D8', 
    'DJ', 'DQ', 'HJ', 'S7']

In [4]:
card_by_card = CardByCard(dealer, vuln, hands, auction, play, models, sampler, False)

Loaded lib dds.dll
DDSolver being loaded - dds mode 1


In [5]:
# calling this starts the analysis
# it will go bid-by-bid and card-by-card, and will take a few moments
# possible mistakes will be annotated with ? or with ?? (if it's a bigger mistake)
# (possible mistake means that the engine does not agree with the bid/play. the engine could be wrong too :))

await card_by_card.analyze()

analyzing the bidding


DDSolver being loaded - dds mode 1


3C OK NN-value: 0.975
PASS OK NN-value: 0.999
4C Suggested bid from NN: CandidateBid(bid=PASS, insta_score=0.8325, expected_score=   -2, expected_mp=---, expected_imp=---, expected_tricks= 8.98, adjust=41.63, alert=  )
4C is not in the bids from the neural network
4H OK NN-value: 0.279
PASS OK NN-value: 0.742
PASS OK NN-value: 0.928
PASS Suggested bid from NN: CandidateBid(bid=5C  , insta_score=0.3402, expected_score= -174, expected_mp=---, expected_imp=---, expected_tricks= 8.65, adjust=17.01, alert=  )
PASS NN-values:CandidateBid(bid=PASS, insta_score=0.6132, expected_score= -268, expected_mp=---, expected_imp=---, expected_tricks= 9.93, adjust=30.66, alert=  )
analyzing opening lead
CA
CA OK
analyzing play


DDSolver being loaded - dds mode 1


C8 Forced
C3 OK
C5 Forced
S8 OK
ST OK
SQ OK
SA ? losing: 0.14
DK OK
DT OK
D6 OK
D3 OK
D9 OK
D7 Forced
Error: Duplicate cards
   ved BGADLL.PIMC.validateInput() i D:\GitHub\BGA\BGADLL\PIMC.cs:linje 152
   ved BGADLL.PIMC.SetupEvaluation(Hand[] our, Hand oppos, Play current_trick, Play previous_tricks, Constraints[] consts, Player nextToLead, Int32 maxPlayout, Boolean autoplaysingleton, Boolean useStratefy) i D:\GitHub\BGA\BGADLL\PIMC.cs:linje 246
max_playout 400
player_i 3
T53..AJ6542. A4.AJ7632.K9.
K986.8.QT873.KQJT97643 QH QH QH KH KH KH
Voids: [set(), set(), set(), set()]
South
Tricks taken 0
min tricks 10
East (RHO) 0 4 0 7 0 0 0 4 0 3
West (LHO) 4 10 0 6 0 2 0 3 2 7
Current trick QH QH QH KH KH KH
Previous tricks AC 8C 2C 5C 7S JS QS 2S 9H 5H 4H TH


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
# the engine does not agree with the 1N opening.
# indeed, it's a little offbeat with a singleton
# let's see what the engine is thinking (what would it bid instead)

card_by_card.bid_responses[0].to_dict()  # the 0 index is the first bid in the auction

the engine very confidently opens `1C` and doesn't even consider `1N`

In [ ]:
# what about the opening lead? let's see...

card_by_card.cards['CA'].to_dict()

the engine agrees with leading a low club, but it's very close. the alternative is a low heart

In [ ]:
# the engine considers dummy's discard of D3 on the first trick a big mistake.
# perhaps we should ruff instead, let's see what the engine suggests

card_by_card.cards['D3'].to_dict()

indeed, the best play is to ruff low.

looking at the samples, we see that East has the `CA` in every sample (this is by inference because underleading an A is very unlikely)